# Project Two, Non-Linear Programming
# Jordan Sahs

----

## Conjugate Gradient Method of Fletcher and Reeves

Benchmark problems were derived from here: https://en.wikipedia.org/wiki/Test_functions_for_optimization

-----

### Note: Must install numbdifftools for taking the gradient

In [1]:
#!pip install numdifftools

## Global Parameters

In [2]:
import numdifftools as nd, math, numpy as np, random as rand

#Global Parameters
e = 0.0001
accuracy = 10
number_of_variables = 10

## Conjugate Gradient Method of Fletcher and Reeves

In [3]:
def Fletcher_Reeves(x, e, func, penalty = False):
    
    def norm(gradient):
        return round(math.sqrt(sum([grad**2 for grad in gradient])), accuracy)
    
    #For Unconstrained Problems, KKT is nothing more than a sub-gradient optimality condition.
    def KKT_Optimality(gradient):
        return np.round(sum(gradient), 0) == 0
    
    
    # Looks for the best region in R to find an optimal solution.
    # The value "a", the lowerbound, is always assumed to be zero,
    # as lambda has to be non-negative.
    def initial_search_region(y, d):
        
        c = 0
        
        while True:
            
            b = math.pow(2, c)
            
            lower = func(y)
            high  = func(y + b*d)
            mid   = func(y + (b/2)*d)
            
            if mid > lower:
                c -= 1
                
            elif mid > high:
                c += 1
                
            else:
                break
                
        return b
    
    
    def golden_ratio(region, y, d):
        
        a, b = region[0], region[1]
        
        lamb = a + (1 - 0.618)*(b - a)
        mu   = a + 0.618*(b - a)
        
        flamb = func(y + lamb*d)
        fmu   = func(y + mu*d)
        
        while b - a > e:
            
            if flamb > fmu:
                
                a = lamb
                lamb = mu
                mu = a + 0.618*(b - a)
                fmu   = func(y + mu*d)
                
            elif flamb <= fmu:
                
                b = mu
                mu = lamb
                lamb = a + (1 - 0.618)*(b - a)
                flamb   = func(y + lamb*d)
                
            
        return (a+b)/2
        
        
            
    #######################################
    # Fletcher and Reeves Algorithm Start
    #######################################
    
    y = np.copy(x)
    
    grad = nd.Gradient(func)(y)
    
    d = -grad
    
    k, j = 1, 1
    
    while norm(grad) > e:
        
        #Solve Minimization Function
        lamb = golden_ratio([0, initial_search_region(y, d)], y, d)
        
        #Assign new y
        y = np.round(y + lamb * d, accuracy)
        
        if j < len(x):
            
            #Calculate new direction
            prev_grad = np.copy(grad)
            grad = nd.Gradient(func)(y)
            
            alpha = sum([g**2 for g in grad])/sum([g**2 for g in prev_grad])

            d = np.round(-grad + alpha*d, accuracy)
            
            j += 1
            
        else:
            j = 1
            k += 1
            
            grad = nd.Gradient(func)(y)
            d = -grad
    
    if not penalty:
        print(f"Close to optimal point: {y}")
        print(f"Value of the function: {round(func(y), accuracy)}")
        print(f"Number of iterations: {k}")
        print(f"Is KKT Optimal? {KKT_Optimality(grad)}")
        
    else: 
        return y

# Problem One: Quasiconvex Function

Sphere Function:

$$f = \sum_{j=0}^n x_i^2$$

Optimality is acieved where $(x_1, \ldots, x_n) = (0,\ldots, 0)$ with $f=0$

In [4]:
def quasiconvex_func(x):
    return sum([i**2 for i in x])

In [5]:
Fletcher_Reeves([500] * number_of_variables, e, quasiconvex_func)

Close to optimal point: [8.0468e-06 8.0468e-06 8.0468e-06 8.0468e-06 8.0468e-06 8.0468e-06
 8.0468e-06 8.0468e-06 8.0468e-06 8.0468e-06]
Value of the function: 6e-10
Number of iterations: 2
Is KKT Optimal? True


In [6]:
Fletcher_Reeves([999] * number_of_variables, e, quasiconvex_func)

Close to optimal point: [2.8714e-06 2.8714e-06 2.8714e-06 2.8714e-06 2.8714e-06 2.8714e-06
 2.8714e-06 2.8714e-06 2.8714e-06 2.8714e-06]
Value of the function: 1e-10
Number of iterations: 2
Is KKT Optimal? True


In [7]:
Fletcher_Reeves([-500] * number_of_variables, e, quasiconvex_func)

Close to optimal point: [-8.0468e-06 -8.0468e-06 -8.0468e-06 -8.0468e-06 -8.0468e-06 -8.0468e-06
 -8.0468e-06 -8.0468e-06 -8.0468e-06 -8.0468e-06]
Value of the function: 6e-10
Number of iterations: 2
Is KKT Optimal? True


# Problem Two: Multi Function

The Shekel function will be used due to the amounts of minima solutions: https://en.wikipedia.org/wiki/Shekel_function

----

The exact function being used is:

$$f = \sum^{m}_{i=0}\left(c_i + \sum_{j=0}^n(x_j - a_{ij})^2\right)^{-1}$$

In [8]:
maxima = 10

c = [rand.randint(0, number_of_variables) for i in range(maxima)]

a = [[rand.randint(0, number_of_variables) for i in range(number_of_variables)] for i in range(maxima)]

def multi_func(x):
    return math.sqrt(sum([1/(c[i] + sum([(x[j]-a[i][j])**2 for j in range(number_of_variables)])) for i in range(maxima)]))

In [9]:
Fletcher_Reeves([99] * number_of_variables, e, multi_func)

Close to optimal point: [99 99 99 99 99 99 99 99 99 99]
Value of the function: 0.0106142444
Number of iterations: 1
Is KKT Optimal? True


In [10]:
Fletcher_Reeves([1500] * number_of_variables, e, multi_func)

Close to optimal point: [1500 1500 1500 1500 1500 1500 1500 1500 1500 1500]
Value of the function: 0.0006688149
Number of iterations: 1
Is KKT Optimal? True


In [11]:
Fletcher_Reeves([-1500] * number_of_variables, e, multi_func)

Close to optimal point: [-1500 -1500 -1500 -1500 -1500 -1500 -1500 -1500 -1500 -1500]
Value of the function: 0.0006645304
Number of iterations: 1
Is KKT Optimal? True


# Extra Benchmarks: Rosenbrock Function

"In mathematical optimization, the Rosenbrock function is a non-convex function, introduced by Howard H. Rosenbrock in 1960,... It is also known as Rosenbrock's valley or Rosenbrock's banana function." - from Wikipedia: https://en.wikipedia.org/wiki/Rosenbrock_function

This is to motivate its constrained use in problem three.

Note that the global minimum is around $x = (1,1,\ldots,1,1)$

-----

The exact function being used:

$$\sum_{i=0}^{n-1}100(x_{i+1} - x_i)^2 + (1-x_i)^2$$

In [12]:
def Rosenbrock_function(x):
    return sum([100*(x[i+1] - x[i])**2 + (1 - x[i])**2 for i in range(len(x)-1)])

In [13]:
Fletcher_Reeves([0] * number_of_variables, e, Rosenbrock_function)

Close to optimal point: [1.00000953 1.00000968 1.00000986 1.00000989 1.00000991 1.0000101
 1.00001045 1.00001067 1.00001076 1.00001081]
Value of the function: 9e-10
Number of iterations: 36
Is KKT Optimal? True


In [14]:
Fletcher_Reeves([50] * number_of_variables, e, Rosenbrock_function)

Close to optimal point: [1.00000254 1.00000266 1.00000269 1.00000267 1.00000276 1.00000273
 1.00000264 1.00000268 1.00000278 1.00000274]
Value of the function: 1e-10
Number of iterations: 69
Is KKT Optimal? True


# Problem Three: Constrained Function

Rosenbrock function constrained with a $n-1$ cubics and lines:

Constaint One:

$$(x_i-1)^3 - x_{i+1}+1\le0\quad\forall i \in [1,\ldots,n-1]$$

Constaint two:

$$x_i+x_{i+1}\le2\quad\forall i \in [1,\ldots,n-1]$$

In [15]:
def constraint_one(x):
    return [(x[i] - 1)**3 - x[i + 1] + 1 for i in range(len(x)-1)]

def constraint_two(x):
    return [x[i] + x[i + 1] - 2 for i in range(len(x)-1)]

In [16]:
def penalty_method(x, u, B, func, constraints):
    
    def pen_func(z):
        return sum([max(0, i) for i in constraints[0](z)])**2 + sum([abs(i) for i in constraints[1](z)])**2
    
    def merged_func(y):
        return func(y) + u*pen_func(y)
    
    k = 0

    while u*pen_func(x) > e:
        
        x = Fletcher_Reeves(x, e, merged_func, True)
        
        u = B*u
        
        k += 1
        
    print(f"Close to optimal point: {x}")
    print(f"Value of the function: {round(func(x), accuracy)}")
    print(f"Number of iterations: {k}")
    
    optimality = True
    
    for c in constraint_one(x):
        if c > e:
            optimality = False
            break
            
    for c in constraint_two(x):
        if c > e or not optimality:
            optimality = False
            break
            
    print(f"Is solution feasible? {optimality}")

In [17]:
penalty_method([0] * number_of_variables, 10, 2, Rosenbrock_function, [constraint_one, constraint_two])

Close to optimal point: [1.00000038 1.00000018 1.00000017 1.00000003 1.00000003 0.99999998
 1.00000004 1.00000006 1.00000005 1.00000019]
Value of the function: 0.0
Number of iterations: 1
Is solution feasible? True


In [18]:
penalty_method([100] * number_of_variables, 10, 2, Rosenbrock_function, [constraint_one, constraint_two])

Close to optimal point: [1.0000001  0.99999999 0.99999995 0.99999986 0.99999998 0.99999998
 1.00000005 1.00000002 1.00000004 0.99999999]
Value of the function: 0.0
Number of iterations: 1
Is solution feasible? True


In [19]:
penalty_method([-100] * number_of_variables, 10, 2, Rosenbrock_function, [constraint_one, constraint_two])

Close to optimal point: [1.00000025 1.00000015 1.00000001 0.99999997 1.00000008 1.00000012
 1.00000001 0.99999998 1.00000004 1.0000001 ]
Value of the function: 0.0
Number of iterations: 1
Is solution feasible? True
